In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline

In [2]:
import os 
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip

import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from exit.utils import get_model, visualize_2motions, generate_src_mask, uniform, cosine_schedule
from einops import rearrange, repeat
import os
import torch.nn.functional as F
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 't2m'
args.down_t = 2

train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

dataset_opt_path = 'checkpoints/kit/Comp_v6_KLD005/opt.txt' if args.dataname == 'kit' else 'checkpoints/t2m/Comp_v6_KLD005/opt.txt'

wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt


100%|██████████| 1460/1460 [00:00<00:00, 1764.26it/s]


Pointer Pointing at 0
Reading checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 28) Completed!!


In [4]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

# https://github.com/openai/CLIP/issues/111
class TextCLIP(torch.nn.Module):
    def __init__(self, model) :
        super(TextCLIP, self).__init__()
        self.model = model
        
    def forward(self,text):
        return self.model.encode_text(text)
clip_model = TextCLIP(clip_model)

In [5]:
args.nb_code = 512 # 8192 # 
args.code_dim = 512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None
args.quantizer = 'ema_reset'
args.mu = 0.99
args.clip_dim = 512
args.block_size = 51
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4


######## LARGE ########
args.num_layers = 9
args.embed_dim_gpt = 1024
# args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/2023-04-17-21-52-42_HML3D_18_lossAllToken_vqLast_big/net_best_fid.pth'
# args.resume_pth = f'/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/VQVAE_official_pretrain/net_best_fid.pth'
# args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/2023-04-17-21-52-42_HML3D_18_lossAllToken_vqLast_big/net_best_fid.pth'
args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/2023-05-16-21-11-20_HML3D_30_endInput_noEndOutput_51Block/net_last.pth'
args.resume_pth = f'/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/VQVAE_official_last/net_last.pth'

######## SMALL ########
# args.num_layers = 8
# args.embed_dim_gpt = 512
# args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/2023-04-15-19-33-09_HML3D_11_lossAllTokenAtOnce_officialVQ/net_last.pth'
# args.resume_pth = f'/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/VQVAE_official_pretrain/net_best_fid.pth'
# args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/2023-04-17-21-53-21_HML3D_17_lossAllToken_vqLast_small/net_best_fid.pth'
# args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/2023-04-17-21-53-21_HML3D_17_lossAllToken_vqLast_small/net_last.pth'
# args.resume_pth = f'/home/epinyoan/git/MaskText2Motion/T2M-BD/output/t2m/VQVAE_official_last/net_last.pth'

if args.dataset_name == 'kit':
    args.vq_name = 'VQVAE'
elif args.dataset_name == 't2m':
    args.vq_name = '2_VQVAE_HML3D'


args.exp_name = 'TEMP'
args.out_dir = 'output_GPT_Final'
args.out_dir = os.path.join(args.out_dir, f'{args.exp_name}')
args.vq_dir= f'output/{args.dataset_name}/{args.vq_name}'
codebook_dir = f'{args.vq_dir}/codebook/'
os.makedirs(args.out_dir, exist_ok = True)
os.makedirs(args.vq_dir, exist_ok = True)
os.makedirs(codebook_dir, exist_ok = True)

net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate)
trans_encoder = trans.Text2Motion_Transformer(num_vq=args.nb_code, 
                                    embed_dim=args.embed_dim_gpt, 
                                    clip_dim=args.clip_dim, 
                                    block_size=args.block_size, 
                                    num_layers=args.num_layers, 
                                    n_head=args.n_head_gpt, 
                                    drop_out_rate=args.drop_out_rate, 
                                    fc_rate=args.ff_rate)

ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
# net = torch.nn.DataParallel(net)

ckpt = torch.load(args.resume_trans, map_location='cpu')
trans_encoder.load_state_dict(ckpt['trans'], strict=True)
trans_encoder = torch.nn.DataParallel(trans_encoder)
net.eval()
net.cuda()
trans_encoder.train()
trans_encoder.cuda()
''

''

### Eval

In [6]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

100%|██████████| 1460/1460 [00:00<00:00, 3119.66it/s]

Pointer Pointing at 0


In [ ]:
# for word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token, name in val_loader:
#     for k in range(pose.shape[0]):
#         x = pose[k].detach().cpu().numpy()
#         l = m_length[k]
#         caption = clip_text[k][:250]
#         np.save(f'study/data/{caption}.npy', pose[k])
#         visualize_2motions(x, val_loader.dataset.std, val_loader.dataset.mean, args.dataname, l, save_path=f'study/data/{caption}.html')

In [7]:
batch = next(iter(val_loader))
word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token, name = batch

In [9]:
pose[0].shape

torch.Size([196, 263])

In [18]:
k = 0
x = pose[k].detach().cpu().numpy()
l = m_length[k]
caption = clip_text[k]
print(caption)
visualize_2motions(x, val_loader.dataset.std, val_loader.dataset.mean, args.dataname, l, save_path='study/data/data.html')

a person runs in a forward direction beginning with their right leg and swaying both left and right arms side to side.
